In [10]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/Downloads/bets/")
bets = bets.to_pandas()
bets.head()

,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
0,9786810,26,DE,RT,Goal,OverUnder,None,FB-107952,2022-09-01,2022-09-01 00:00:07.176,...,INT,Football,Friendly,60.0,11.25,5,2.98,6.25,6.25,6.56
1,9187810,26,DE,SET1,Game,LeadAfter,None,TN-21815,2022-09-01,2022-09-01 00:00:20.216,...,US,Tennis,US Open Women,0.0,5.00,5,3.56,3.02,3.25,2.94
2,9187810,26,DE,SET5,Game,NthPoint2w,None,TN-21771,2022-09-01,2022-09-01 00:00:20.216,...,US,Tennis,US Open Men,0.0,5.00,5,3.58,5.00,3.25,3.26
3,12072510,44,DE,HT1,Goal,OverUnder,None,FB-106228,2022-09-01,2022-09-01 00:00:22.536,...,US,Football,MLS,0.0,3.75,5,3.29,3.75,3.75,3.75
4,12072510,44,DE,RT,Goal,OverUnder,None,FB-106190,2022-09-01,2022-09-01 00:00:22.536,...,UY,Football,Cup,0.0,3.75,5,3.26,3.75,3.75,3.75


In [2]:
from sklearn.preprocessing import LabelEncoder

categorical_features = ['country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league']

le = LabelEncoder()

# Convert the variables to numerical
for i in range(len(categorical_features)):
    label = le.fit_transform(bets[categorical_features[i]])
    bets[categorical_features[i]] = label
bets.head()


,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
0,9786810,26,2,37,7,12,2320,15531,2022-09-01,2022-09-01 00:00:07.176,...,61,13,618,60.0,11.25,5,2.98,6.25,6.25,6.56
1,9187810,26,2,39,6,7,2320,63387,2022-09-01,2022-09-01 00:00:20.216,...,130,28,1484,0.0,5.00,5,3.56,3.02,3.25,2.94
2,9187810,26,2,86,6,9,2320,63380,2022-09-01,2022-09-01 00:00:20.216,...,130,28,1481,0.0,5.00,5,3.58,5.00,3.25,3.26
3,12072510,44,2,10,7,12,2320,15484,2022-09-01,2022-09-01 00:00:22.536,...,130,13,919,0.0,3.75,5,3.29,3.75,3.75,3.75
4,12072510,44,2,37,7,12,2320,15476,2022-09-01,2022-09-01 00:00:22.536,...,132,13,415,0.0,3.75,5,3.26,3.75,3.75,3.75


In [3]:
# Create features and target
XX = bets[['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
X = bets[['age', 'country', 'section', 'point', 'market', 'region', 'sport', 'league', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
y = bets[['stake_eur']]

# show rows with NaN values
# X[X.isna().any(axis=1)]

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101, shuffle=False)

#print('train: from', X_train['placement_date'].min(), ' to ', X_train['placement_date'].max())
#print('train: from', X_test['placement_date'].min(), ' to ', X_test['placement_date'].max())

# Train the model
regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
regr.fit(X_train, y_train.values.ravel())

RandomForestRegressor(max_depth=10, n_estimators=10, random_state=101)

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = regr.predict(X_test)

result = X_test
result['stake_eur'] = y_test
result['prediction'] = predictions.tolist()
result.head()

,age,country,section,point,market,region,sport,league,weekday_stake,section_2w,point_2w,market_2w,stake_eur,prediction
3897318,31,2,37,7,12,96,13,804,1.55,1.5,1.5,1.55,0.11,1.563730
3897319,31,2,37,7,12,96,13,804,1.68,1.5,1.5,1.55,0.11,1.563730
3897320,31,2,37,7,19,96,13,804,1.52,1.5,1.5,1.89,0.11,1.795443
3897321,31,2,37,7,19,96,13,804,1.44,1.5,1.5,1.89,0.11,1.768160
3897322,31,2,37,7,19,96,13,804,1.52,1.5,1.5,1.89,0.11,1.795443


In [9]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

# Mean squared error (MSE)
mse = mean_squared_error(y_test.values.ravel(), predictions)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)

# Print metrics
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

Mean Absolute Error: 5.02
Mean Squared Error: 892.91
R-squared scores: 0.59


In [6]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
parameters = {
    'max_depth': [70, 80, 90, 100],
    'n_estimators': [900, 1000, 1100]
}
gridforest = GridSearchCV(regr, parameters, cv = 3, n_jobs = -1, verbose = 1)
gridforest.fit(X_train, y_train)
gridforest.best_params_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages

KeyboardInterrupt: 

In [8]:
# Get features list
characteristics = X.columns
# Get the variables importances, sort them, and print the result
importances = list(regr.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances];

Variable: section_2w           Importance: 0.72
Variable: market_2w            Importance: 0.12
Variable: weekday_stake        Importance: 0.11
Variable: point_2w             Importance: 0.02
Variable: age                  Importance: 0.01
Variable: league               Importance: 0.01
Variable: country              Importance: 0.0
Variable: section              Importance: 0.0
Variable: point                Importance: 0.0
Variable: market               Importance: 0.0
Variable: region               Importance: 0.0
Variable: sport                Importance: 0.0


In [ ]:
# Import library for visualization
import matplotlib.pyplot as plt
# Visualize the variables importances
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Importance')
plt.xlabel('Variable')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()